In [1]:
from os.path import join
import numpy as np
from scipy import optimize
from microstruktur.signal_models import three_dimensional_models
from microstruktur.signal_models import gradient_conversions

In [31]:
from importlib import reload
reload(three_dimensional_models)

<module 'microstruktur.signal_models.three_dimensional_models' from '/Users/dwasserm/sources/microstruktur/microstruktur/signal_models/three_dimensional_models.py'>

In [2]:
bvals = np.loadtxt(join(three_dimensional_models.GRADIENT_TABLES_PATH, 'bvals_hcp_wu_minn.txt'))
gradient_directions = np.loadtxt(join(three_dimensional_models.GRADIENT_TABLES_PATH, 'bvecs_hcp_wu_minn.txt'))

In [3]:
stick = three_dimensional_models.I1Stick(lambda_par=2e-3, mu=[0, 0])

In [4]:
E = stick(bvals, gradient_directions)
E

array([ 1.        ,  0.9335884 ,  0.60888331,  0.00647561,  0.72635583,
        0.41434429,  0.32360809,  0.19819166,  0.09504999,  0.89685335,
        0.71998388,  0.02296437,  0.5116401 ,  0.23070807,  0.99901891,
        0.87608816,  1.        ,  0.77699837,  0.92441009,  0.00726072,
        0.10433723,  0.45190814,  0.24258347,  0.87642082,  0.11050229,
        0.03680408,  0.30671406,  0.25048305,  0.43244879,  0.80229888,
        0.86910295,  0.11561769,  1.        ,  0.19698608,  0.97595931,
        0.84296755,  0.16495121,  0.05832137,  0.52947699,  0.69191079,
        0.0049633 ,  0.95691371,  0.0384834 ,  0.60712385,  0.07825937,
        0.99847135,  0.43527201,  0.98953406,  1.        ,  0.2784147 ,
        0.70151996,  0.79582974,  0.82327515,  0.32211598,  0.02482972,
        0.29564166,  0.04263531,  0.59804174,  0.74817085,  0.23847658,
        0.09969115,  0.99720355,  0.06091185,  0.70564666,  1.        ,
        0.14599592,  0.07093692,  0.99862363,  0.99930743,  0.38

In [5]:
x0 = stick.parameters_to_parameter_vector(mu=[np.pi/4, 0], lambda_par=2e-3)
res = optimize.minimize(
    stick.objective_function, x0, (bvals, gradient_directions, E),
    bounds=stick.bounds_for_optimization, options={'disp': True}
)
res

      fun: 2.2193222037573134e-14
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([  1.03347964e-04,   5.88763873e-09,   0.00000000e+00])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 212
      nit: 9
   status: 2
  success: False
        x: array([ 0.002     ,  0.        , -0.02770102])

In [6]:
stick.parameter_vector_to_parameters(res.x)

{'lambda_par': array([ 0.002]), 'mu': array([ 0.        , -0.02770102])}

In [8]:
stick2 = three_dimensional_models.I1Stick(lambda_par=2e-3, mu=[np.pi/4, 0])

# Linked Parameter Example
lambda_par of the second stick is double that of the first stick

In [245]:
reload(three_dimensional_models)

<module 'microstruktur.signal_models.three_dimensional_models' from '/Users/dwasserm/sources/microstruktur/microstruktur/signal_models/three_dimensional_models.py'>

In [247]:
def double_scalar_plus_constant(x, c):
    return 2 * x + c

parameter_links = [
    (
        stick2, 'lambda_par',
        double_scalar, [
            (stick, 'lambda_par'),
            (None, 'partial_volume_0')
        ]
    )
]

two_sticks = three_dimensional_models.PartialVolumeCombinedMicrostrukturModel(
    [stick, stick2], parameter_links=parameter_links, 
    optimise_partial_volumes=True
)
two_sticks.parameter_cardinality

OrderedDict([('I1Stick_2_mu', 2),
             ('I1Stick_1_lambda_par', 1),
             ('I1Stick_1_mu', 2),
             ('partial_volume_0', 1)])

In [248]:
#Check if we add the linked parameter correctly
two_sticks.add_linked_parameters_to_parameters(two_sticks.parameter_defaults)

TypeError: double_scalar() takes 1 positional argument but 2 were given

In [229]:
E = two_sticks(bvals, gradient_directions, **two_sticks.parameter_defaults)
E

array([ 1.        ,  0.62664888,  0.38100631,  0.00482471,  0.82305019,
        0.68540504,  0.16294488,  0.13038191,  0.51436428,  0.44943023,
        0.8282815 ,  0.01351423,  0.25594158,  0.47406377,  0.51239407,
        0.89357478,  1.        ,  0.41274663,  0.79231092,  0.00592494,
        0.4709468 ,  0.26023052,  0.3872915 ,  0.72104735,  0.3379752 ,
        0.01840627,  0.58349936,  0.1256159 ,  0.24573178,  0.55127555,
        0.50310335,  0.54497333,  1.        ,  0.11294252,  0.66466635,
        0.68186532,  0.12204689,  0.03549749,  0.34738607,  0.36189618,
        0.05892657,  0.93414221,  0.0192688 ,  0.68980665,  0.04928554,
        0.99919599,  0.51851628,  0.49551824,  1.        ,  0.28602312,
        0.35185865,  0.87635293,  0.42289467,  0.36321367,  0.06546572,
        0.15745213,  0.39052861,  0.33674528,  0.78891668,  0.59071318,
        0.12887812,  0.78468649,  0.03051917,  0.36078926,  1.        ,
        0.21356997,  0.03572172,  0.52358079,  0.57498484,  0.48

In [241]:
x0 = two_sticks.parameters_to_parameter_vector(
    **two_sticks.parameter_defaults
)
x0

array([ 0.78539816,  0.        ,  0.002     ,  0.        ,  0.        ,
        0.5       ])

In [242]:
x0 = two_sticks.parameters_to_parameter_vector(
    I1Stick_1_mu=[0, 0], I1Stick_1_lambda_par=0e-3,
    I1Stick_2_mu=[0, 0], #I1Stick_2_lambda_par=2e-3,
    partial_volume_0=.2
)
res = optimize.minimize(
    two_sticks.objective_function, x0, (bvals, gradient_directions, E),
    bounds=two_sticks.bounds_for_optimization, options={'disp': True}
)
res

      fun: 8.5811899386455722e-11
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([  6.20431517e-07,  -2.41488623e-06,   1.92186194e-05,
         2.80716327e-06,  -1.88836455e-11,   2.42725814e-06])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 595
      nit: 52
   status: 0
  success: True
        x: array([  7.85413242e-01,  -1.34223621e-05,   2.00002275e-03,
         2.96314423e-05,   1.63732510e-04,   5.00020029e-01])

In [243]:
two_sticks.parameter_vector_to_parameters(res.x)

{'I1Stick_1_lambda_par': array([ 0.00200002]),
 'I1Stick_1_mu': array([  2.96314423e-05,   1.63732510e-04]),
 'I1Stick_2_mu': array([  7.85413242e-01,  -1.34223621e-05]),
 'partial_volume_0': array([ 0.50002003])}

In [244]:
two_sticks.add_linked_parameters_to_parameters(two_sticks.parameter_vector_to_parameters(res.x))

{'I1Stick_1_lambda_par': array([ 0.00200002]),
 'I1Stick_1_mu': array([  2.96314423e-05,   1.63732510e-04]),
 'I1Stick_2_lambda_par': array([ 0.00400005]),
 'I1Stick_2_mu': array([  7.85413242e-01,  -1.34223621e-05]),
 'partial_volume_0': array([ 0.50002003])}